In [1]:
import requests
import os
from dotenv import load_dotenv
import sys
load_dotenv()
src_dir = os.path.dirname(os.getcwd()) + "\\src"
sys.path.insert(0, src_dir)
from utils.helper_functions import gather_article_set, BASE_NYT_URL, gen_stock_filter, gen_mkt_filter, make_api_call
import pandas as pd
import numpy as np

key = os.getenv("NYT_API_KEY") # get personal API key from environment variables

#### Specific Stock Article calls
- 'fq': '(headline:("{stock_name}") AND body:("{stock_name}")) AND news_desk:("Business", "Financial")'

#### General Market Calls
- ??
- (headline:("{keyword1}") AND body:("keyword1")) OR (headline:("keyword2") AND body:("keyword2"))... AND news_desk:("Business", "Financial")



In [3]:
market_keywords = [
    'S&P', 'Stock Market', 'Wall Street'
]

In [9]:
gen_stock_filter(stock_name = "Microsoft", ticker = "MSFT")

'(headline:("Microsoft") AND body:("Microsoft")) OR (body:("MSFT"))'

In [10]:
gen_stock_filter(stock_name = "Nike", ticker = "NKE")

'(headline:("Nike") AND body:("Nike")) OR (body:("NKE"))'

In [11]:
gen_mkt_filter('S&P','Inflation')

'(headline:("S&P") AND body:("S&P")) OR (headline:("Inflation") AND body:("Inflation")) AND news_desk:("Business", "Financial")'

In [12]:
gen_mkt_filter(*market_keywords)

'(headline:("S&P") AND body:("S&P")) OR (headline:("Stock Market") AND body:("Stock Market")) OR (headline:("Wall Street") AND body:("Wall Street")) AND news_desk:("Business", "Financial")'

In [4]:
market_keywords

['S&P', 'Stock Market', 'Wall Street']

In [32]:
# Example call to get Nike Articles between January and May 2022
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json" # API base url

payload = {
    # 'fq': 'body:("Apple AND stock")',
    'fq': gen_stock_filter("Nike",'NKE'),
    'begin_date': "2022-01-01",
    'end_date': "2022-05-31",
    'api-key': key,
    'fl': 'lead_paragraph,snippet,abstract,pub_date,headline,web_url,source',
    'page': 0
}

resp = requests.get(url, params=payload) # make GET requests to API
print(resp.status_code) # 200 status code success

print("Hits: ",resp.json()['response']['meta']['hits']) # get the number of article hits

200
Hits:  3


In [33]:
docs = resp.json()['response']['docs'] # Get the docs from response


In [34]:
docs[0]

{'abstract': 'The Air Force 1, manufactured in Italy, will be unveiled in a new installation in Brooklyn. It was one of the designer’s final projects before his death.',
 'web_url': 'https://www.nytimes.com/2022/05/20/style/louis-vuitton-nike-virgil-abloh.html',
 'snippet': 'The Air Force 1, manufactured in Italy, will be unveiled in a new installation in Brooklyn. It was one of the designer’s final projects before his death.',
 'lead_paragraph': 'Over the years, Virgil Abloh approached his work with Nike as something slightly different than a straightforward collaborator. He was a tinkerer. A nudger. A questioner. Often his ideas verged on the raw: What if you punch holes in the sneaker? What can we print on it? What if we rendered the invisible parts of construction visible?',
 'source': 'The New York Times',
 'headline': {'main': 'Louis Vuitton Meets Nike in Virgil Abloh’s Dream Sneaker',
  'kicker': None,
  'content_kicker': None,
  'print_headline': 'Virgil Abloh’s Dream Sneaker, 

In [15]:
resp.json()['response']['docs'][0]

{'abstract': 'The Air Force 1, manufactured in Italy, will be unveiled in a new installation in Brooklyn. It was one of the designer’s final projects before his death.',
 'snippet': 'The Air Force 1, manufactured in Italy, will be unveiled in a new installation in Brooklyn. It was one of the designer’s final projects before his death.',
 'lead_paragraph': 'Over the years, Virgil Abloh approached his work with Nike as something slightly different than a straightforward collaborator. He was a tinkerer. A nudger. A questioner. Often his ideas verged on the raw: What if you punch holes in the sneaker? What can we print on it? What if we rendered the invisible parts of construction visible?',
 'headline': {'main': 'Louis Vuitton Meets Nike in Virgil Abloh’s Dream Sneaker',
  'kicker': None,
  'content_kicker': None,
  'print_headline': 'Virgil Abloh’s Dream Sneaker, Times 47',
  'name': None,
  'seo': None,
  'sub': None},
 'pub_date': '2022-05-20T09:00:39+0000'}

In [16]:
# function to see an example article when given an index
def see_example(num):
    if resp.json()['response']['meta']['hits'] == 0:
        return "No hits"
    example = docs[num]
    print(example['abstract']) 
    print(example['snippet'])
    print(example['lead_paragraph'])
    print(example['headline'])


In [17]:
see_example(0)


The Air Force 1, manufactured in Italy, will be unveiled in a new installation in Brooklyn. It was one of the designer’s final projects before his death.
The Air Force 1, manufactured in Italy, will be unveiled in a new installation in Brooklyn. It was one of the designer’s final projects before his death.
Over the years, Virgil Abloh approached his work with Nike as something slightly different than a straightforward collaborator. He was a tinkerer. A nudger. A questioner. Often his ideas verged on the raw: What if you punch holes in the sneaker? What can we print on it? What if we rendered the invisible parts of construction visible?
{'main': 'Louis Vuitton Meets Nike in Virgil Abloh’s Dream Sneaker', 'kicker': None, 'content_kicker': None, 'print_headline': 'Virgil Abloh’s Dream Sneaker, Times 47', 'name': None, 'seo': None, 'sub': None}


In [18]:
see_example(1)

Barbara Mack hugged Larry Miller after he apologized for killing her brother, Edward David White. But if she were 30 years younger, she told him, “I would have been across that table at you.”
Barbara Mack hugged Larry Miller after he apologized for killing her brother, Edward David White. But if she were 30 years younger, she told him, “I would have been across that table at you.”
PHILADELPHIA — Barbara Mack lives a block from the corner where her teenage brother was shot to death while on his way home from work on Sept. 30, 1965. Hasan Adams was only 8 months old when the bullet took his father. Azizah Arline was months from being born and never met her dad. More than half a century later, they have finally received an apology from, and discussed restitution with, the man who committed the murder.
{'main': 'A Nike Executive Seeks a Family’s Forgiveness for a 1965 Murder', 'kicker': None, 'content_kicker': None, 'print_headline': 'Nike Executive Seeks  Forgiveness of Family   Of ’65 Mu

In [19]:
see_example(-1)

Why would anyone spend that much for a virtual shoe?
Why would anyone spend that much for a virtual shoe?
The market for collectible sneakers has skyrocketed in recent years. And until recently, so had the market for NFTs, or nonfungible tokens, which function as digital certificates of ownership for works of art as well as tattoo designs and virtual real estate.
{'main': 'Nike Sold an NFT Sneaker for $134,000', 'kicker': None, 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}


In [48]:
# Example API call to get general market articles betwwn January and May 2022

payload = {
    # 'fq': 'body:("Apple AND stock")',
    'fq': gen_mkt_filter(*market_keywords),
    'begin_date': "2022-01-01",
    'end_date': "2022-05-31",
    'api-key': key,
    'fl': 'lead_paragraph,snippet,abstract,pub_date,headline',
    'page': 0
}

resp = requests.get(url, params=payload) # make GET requests to API
print(resp.status_code) # 200 status code success
print("Hits: ",resp.json()['response']['meta']['hits']) # get the number of article hits

200
Hits:  25


In [256]:
see_example(0)

The Air Force 1, manufactured in Italy, will be unveiled in a new installation in Brooklyn. It was one of the designer’s final projects before his death.
The Air Force 1, manufactured in Italy, will be unveiled in a new installation in Brooklyn. It was one of the designer’s final projects before his death.
Over the years, Virgil Abloh approached his work with Nike as something slightly different than a straightforward collaborator. He was a tinkerer. A nudger. A questioner. Often his ideas verged on the raw: What if you punch holes in the sneaker? What can we print on it? What if we rendered the invisible parts of construction visible?
{'main': 'Louis Vuitton Meets Nike in Virgil Abloh’s Dream Sneaker', 'kicker': None, 'content_kicker': None, 'print_headline': 'Virgil Abloh’s Dream Sneaker, Times 47', 'name': None, 'seo': None, 'sub': None}


In [257]:
see_example(1)

Barbara Mack hugged Larry Miller after he apologized for killing her brother, Edward David White. But if she were 30 years younger, she told him, “I would have been across that table at you.”
Barbara Mack hugged Larry Miller after he apologized for killing her brother, Edward David White. But if she were 30 years younger, she told him, “I would have been across that table at you.”
PHILADELPHIA — Barbara Mack lives a block from the corner where her teenage brother was shot to death while on his way home from work on Sept. 30, 1965. Hasan Adams was only 8 months old when the bullet took his father. Azizah Arline was months from being born and never met her dad. More than half a century later, they have finally received an apology from, and discussed restitution with, the man who committed the murder.
{'main': 'A Nike Executive Seeks a Family’s Forgiveness for a 1965 Murder', 'kicker': None, 'content_kicker': None, 'print_headline': 'Nike Executive Seeks  Forgiveness of Family   Of ’65 Mu

In [258]:
see_example(-1)

Why would anyone spend that much for a virtual shoe?
Why would anyone spend that much for a virtual shoe?
The market for collectible sneakers has skyrocketed in recent years. And until recently, so had the market for NFTs, or nonfungible tokens, which function as digital certificates of ownership for works of art as well as tattoo designs and virtual real estate.
{'main': 'Nike Sold an NFT Sneaker for $134,000', 'kicker': None, 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}
